# Аналитика ДТП по предоставленным данным

Заказчиком (Оунером задачи) выступает проект «Карта ДТП» https://dtp-stat.ru/ — некоммерческий проект, посвященный проблеме дорожно-транспортных происшествий в России. Это платформа сбора данных о ДТП, бесплатный и открытый сервис аналитики ДТП.

**Цель проекта** - провести глубокий анализ данных, сформулировать и проверить гипотезы, опираясь на доступные признаки.

**Задачи**
* Провести исследовательский анализ данных о ДТП за 2025 год.
* Сформулировать и проверить не менее трех гипотез, основываясь на имеющихся признаках.
* Провести анализ ДТП по всем датасетам, представленным на сайте.
* Построить дашборд используя любой удобный инструмент, с учетом того, что дашборд может быть опубликован

**Декомпозиция работы:**
* Загрузка датасета и знакомство с данными
* Предобработка данных
* Исследовательский анализ данных
* Проверка гипотез
* Общие выводы
* Подготовка дашборда

## 1. Загрузка датасета и знакомство с данными

Загрузим необходимые библиотеки:

In [1]:
#!pip install geopandas

import os
import geopandas as gpd
import pandas as pd
from shapely import wkt

# Настройка отображения Jupiter
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)  # Отображать все строки
pd.set_option('display.max_columns', None)  # Отображать все столбцы
pd.set_option('display.width', None)  # Автоматическая ширина
pd.set_option('display.max_colwidth', None)  # Отображать полную ширину столбцов

**Для скачивания датасетов с сайта - необходимо использовать скрипт из ноутбука "DataLoader.ipynb"**

Загрузим данные:

In [2]:
all_records = []

DOWNLOAD_DIR = 'geojson_files'

# перебираем все .geojson в папке
for filename in os.listdir(DOWNLOAD_DIR):
    if filename.endswith(".geojson"):
        path = os.path.join(DOWNLOAD_DIR, filename)
        try:
            with open(path, encoding="utf-8") as f:
                data = json.load(f)
            accidents = data["features"]

            records = []
            for feature in accidents:
                props = feature["properties"].copy()
                props["geometry"] = feature["geometry"]
                records.append(props)

            all_records.extend(records)
            print(f"Загружено {len(records)} строк из {filename}")

        except Exception as e:
            print(f"Ошибка при обработке {filename}: {e}")

# объединяем в один датафрейм
data = pd.DataFrame(all_records)
print(f"Итого строк во всех файлах: {len(data)}")

Загружено 10307 строк из novgorodskaia-oblast.geojson
Загружено 27892 строк из tiumenskaia-oblast.geojson
Загружено 5855 строк из sevastopol.geojson
Загружено 41694 строк из cheliabinskaia-oblast.geojson
Загружено 7015 строк из respublika-kareliia.geojson
Загружено 25787 строк из volgogradskaia-oblast.geojson
Загружено 20977 строк из vladimirskaia-oblast.geojson
Загружено 2323 строк из magadanskaia-oblast.geojson
Загружено 10181 строк из kurganskaia-oblast.geojson
Загружено 29859 строк из sverdlovskaia-oblast.geojson
Загружено 16533 строк из respublika-dagestan.geojson
Загружено 5274 строк из respublika-tyva.geojson
Загружено 2589 строк из chechenskaia-respublika.geojson
Загружено 12534 строк из arkhangelskaia-oblast.geojson
Загружено 5254 строк из respublika-kalmykiia.geojson
Загружено 13438 строк из tambovskaia-oblast.geojson
Загружено 28453 строк из omskaia-oblast.geojson
Загружено 6748 строк из kabardino-balkarskaia-respublika.geojson
Загружено 4274 строк из iamalo-nenetskii-avtono

Откроем датасет и посмотрим на данные:

In [3]:
data.head()

,id,tags,light,point,nearby,region,scheme,address,weather,category,datetime,severity,vehicles,dead_count,participants,injured_count,parent_region,road_conditions,participants_count,participant_categories,geometry
0,1504730,[Дорожно-транспортные происшествия],Светлое время суток,"{'lat': 58.332904, 'long': 30.415138}",[],Шимский район,200,"Медведь-Батецкий, 7 км",[Ясно],Столкновение,2025-05-01 12:55:00,С погибшими,"[{'year': 2012, 'brand': 'KIA', 'color': 'Серый', 'model': 'Venga', 'category': 'В-класс (малый) до 3,9 м', 'participants': [{'role': 'Водитель', 'gender': 'Женский', 'violations': ['Выезд на полосу встречного движения'], 'health_status': 'Не пострадал', 'years_of_driving_experience': 4}, {'role': 'Пассажир', 'gender': 'Женский', 'violations': [], 'health_status': 'Не пострадал', 'years_of_driving_experience': None}]}, {'year': 2013, 'brand': 'Прочие марки мотоциклов', 'color': 'Красный', 'model': 'Прочие марки мотоциклов', 'category': 'Мопеды с двигателем внутреннего сгорания менее 50 см. куб.', 'participants': [{'role': 'Водитель', 'gender': 'Мужской', 'violations': ['Управление ТС лицом, не имеющим права на управление ТС'], 'health_status': 'Скончался на месте ДТП до приезда скорой медицинской помощи', 'years_of_driving_experience': None}]}]",1,[],0,Новгородская область,[Сухое],3,"[Все участники, Мотоциклисты]","{'type': 'Point', 'coordinates': [30.415138, 58.332904]}"
1,156565,[Дорожно-транспортные происшествия],Светлое время суток,"{'lat': None, 'long': None}",[],Шимский район,600,"Батецкий-Медведь, 10 км",[Ясно],Опрокидывание,2015-06-11 16:10:00,Легкий,"[{'year': 2011, 'brand': 'KIA', 'color': 'Синий', 'model': 'Rio', 'category': 'D-класс (средний) до 4,6 м', 'participants': [{'role': 'Водитель', 'gender': 'Женский', 'violations': ['Другие нарушения ПДД водителем'], 'health_status': 'Раненый, находящийся (находившийся) на амбулаторном лечении, либо которому по характеру полученных травм обозначена необходимость амбулаторного лечения (вне зависимости от его фактического прохождения)', 'years_of_driving_experience': 10}]}]",0,[],1,Новгородская область,[Сухое],1,[Все участники],"{'type': 'Point', 'coordinates': [None, None]}"
2,156274,[Дорожно-транспортные происшествия],"В темное время суток, освещение включено","{'lat': 58.295429, 'long': 30.494614}","[Жилые дома индивидуальной застройки, Нерегулируемый пешеходный переход, Остановка общественного транспорта, Нерегулируемый перекрёсток]",Шимский район,840,"д Старый Медведь, Медведь-Батецкий, 1 км",[Ясно],Наезд на пешехода,2022-10-07 20:30:00,Тяжёлый,"[{'year': 2009, 'brand': 'ВАЗ', 'color': 'Черный', 'model': 'Priora', 'category': 'В-класс (малый) до 3,9 м', 'participants': [{'role': 'Водитель', 'gender': 'Мужской', 'violations': ['Оставление места ДТП', 'Несоответствие скорости конкретным условиям движения'], 'health_status': 'Не пострадал', 'years_of_driving_experience': 13}]}]",0,"[{'role': 'Пешеход', 'gender': 'Мужской', 'violations': [], 'health_status': 'Раненый, находящийся (находившийся) на стационарном лечении'}]",1,Новгородская область,[Сухое],2,"[Все участники, Пешеходы]","{'type': 'Point', 'coordinates': [30.494614, 58.295429]}"
3,156238,[Дорожно-транспортные происшествия],"В темное время суток, освещение включено","{'lat': 58.210136, 'long': 30.721529}","[Административные здания, Многоквартирные жилые дома, Нерегулируемый пешеходный переход]",Шимский район,740,"рп Шимск, ул Новгородская, 6",[Пасмурно],Наезд на пешехода,2022-12-06 17:05:00,Легкий,"[{'year': 2009, 'brand': 'CHEVROLET', 'color': 'Синий', 'model': 'Cruze', 'category': 'D-класс (средний) до 4,6 м', 'participants': [{'role': 'Водитель', 'gender': 'Мужской', 'violations': ['Непредоставление преимущества в движении пешеходу', 'Другие нарушения ПДД водителем'], 'health_status': 'Не пострадал', 'years_of_driving_experience': 25}]}]",0,"[{'role': 'Пешеход', 'gender': 'Женский', 'violations': [], 'health_status': 'Раненый, находящийся (находившийся) на амбулаторном лечении, либо в условиях дневно

Посмотрим общую информацию о датасете:

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1440070 entries, 0 to 1440069
Data columns (total 21 columns):
 #   Column                  Non-Null Count    Dtype 
---  ------                  --------------    ----- 
 0   id                      1440070 non-null  int64 
 1   tags                    1440070 non-null  object
 2   light                   1440070 non-null  object
 3   point                   1440070 non-null  object
 4   nearby                  1440070 non-null  object
 5   region                  1440070 non-null  object
 6   scheme                  1361453 non-null  object
 7   address                 1364914 non-null  object
 8   weather                 1440070 non-null  object
 9   category                1440070 non-null  object
 10  datetime                1440070 non-null  object
 11  severity                1440070 non-null  object
 12  vehicles                1440070 non-null  object
 13  dead_count              1440070 non-null  int64 
 14  participants      

In [5]:
print(f'Исходный датасет содержит {data.shape[0]} записей и {data.shape[1]} признак(а/ов):')

Исходный датасет содержит 1440070 записей и 21 признак(а/ов):


* `id`: идентификатор
* `light`: время суток
* `point`: координаты (гео)
* `nearby`: комментарий по месту происшествия (где произошло)
* `region`: город/район
* `address`: адрес
* `weather`: погода
* `category`: тип ДТП
* `datetime`: дата и время происшествия
* `severity`: тяжесть ДТП/вред здоровью
* `vehicles`: участники – транспортные средства:
    * `year`: год производства транспортного средства
    * `brand`: марка транспортного средства
    * `color`: цвет транспортного средства
    * `model`: модель транспортного средства
    * `category`: категория транспортного средства
    * `role`: роль участника
    * `gender`: пол участника
    * `violations`: нарушения правил участником
    * `health_status`: состояние здоровья участника
    * `years_of_driving_experience`: стаж вождения участника (только у водителей)
* `dead_count`: кол-во погибших в ДТП
* `participants`: участники без транспортных средств (описание, как у участников внутри транспортных средств)
* `injured_count`: кол-во раненых в ДТП
* `parent_region`: родительский регион
* `road_conditions`: состояние дорожного покрытия
* `participants_count`: кол-во участников ДТП
* `participant_categories`: категории участников

## 2. Предобработка данных

Задачи данного раздела:

* Привести типы данных в соответствие
* Проверить пропуски в датасете и обработать их
* Проверить наличие явных и неявных дубликатов и обработать их
* По возможности дообогатить данные на основе имеющихся

### 2.1. Подготовка полей

#### 2.1.1 Преобразование списочных полей к текстовому типу

В полях `tags`, `nearby`, `weather`, `vehicles`, `participants`, `road_conditions`, `participant_categories` содержатся списки, преобразуем данные поля к текстовому типу.

Определим перечень полей для преобразования:

In [6]:
list_features = ['tags', 'nearby', 'weather', 'road_conditions', 'participant_categories']

Приведем поля к текстовому типу:

In [7]:
for column in list_features:
    data[column] = data[column].apply(lambda x: x[0] if len(x) == 1 else ', '.join(x) if isinstance(x, list) else None)

Проверим результат:

In [8]:
data.head()

,id,tags,light,point,nearby,region,scheme,address,weather,category,datetime,severity,vehicles,dead_count,participants,injured_count,parent_region,road_conditions,participants_count,participant_categories,geometry
0,1504730,Дорожно-транспортные происшествия,Светлое время суток,"{'lat': 58.332904, 'long': 30.415138}",,Шимский район,200,"Медведь-Батецкий, 7 км",Ясно,Столкновение,2025-05-01 12:55:00,С погибшими,"[{'year': 2012, 'brand': 'KIA', 'color': 'Серый', 'model': 'Venga', 'category': 'В-класс (малый) до 3,9 м', 'participants': [{'role': 'Водитель', 'gender': 'Женский', 'violations': ['Выезд на полосу встречного движения'], 'health_status': 'Не пострадал', 'years_of_driving_experience': 4}, {'role': 'Пассажир', 'gender': 'Женский', 'violations': [], 'health_status': 'Не пострадал', 'years_of_driving_experience': None}]}, {'year': 2013, 'brand': 'Прочие марки мотоциклов', 'color': 'Красный', 'model': 'Прочие марки мотоциклов', 'category': 'Мопеды с двигателем внутреннего сгорания менее 50 см. куб.', 'participants': [{'role': 'Водитель', 'gender': 'Мужской', 'violations': ['Управление ТС лицом, не имеющим права на управление ТС'], 'health_status': 'Скончался на месте ДТП до приезда скорой медицинской помощи', 'years_of_driving_experience': None}]}]",1,[],0,Новгородская область,Сухое,3,"Все участники, Мотоциклисты","{'type': 'Point', 'coordinates': [30.415138, 58.332904]}"
1,156565,Дорожно-транспортные происшествия,Светлое время суток,"{'lat': None, 'long': None}",,Шимский район,600,"Батецкий-Медведь, 10 км",Ясно,Опрокидывание,2015-06-11 16:10:00,Легкий,"[{'year': 2011, 'brand': 'KIA', 'color': 'Синий', 'model': 'Rio', 'category': 'D-класс (средний) до 4,6 м', 'participants': [{'role': 'Водитель', 'gender': 'Женский', 'violations': ['Другие нарушения ПДД водителем'], 'health_status': 'Раненый, находящийся (находившийся) на амбулаторном лечении, либо которому по характеру полученных травм обозначена необходимость амбулаторного лечения (вне зависимости от его фактического прохождения)', 'years_of_driving_experience': 10}]}]",0,[],1,Новгородская область,Сухое,1,Все участники,"{'type': 'Point', 'coordinates': [None, None]}"
2,156274,Дорожно-транспортные происшествия,"В темное время суток, освещение включено","{'lat': 58.295429, 'long': 30.494614}","Жилые дома индивидуальной застройки, Нерегулируемый пешеходный переход, Остановка общественного транспорта, Нерегулируемый перекрёсток",Шимский район,840,"д Старый Медведь, Медведь-Батецкий, 1 км",Ясно,Наезд на пешехода,2022-10-07 20:30:00,Тяжёлый,"[{'year': 2009, 'brand': 'ВАЗ', 'color': 'Черный', 'model': 'Priora', 'category': 'В-класс (малый) до 3,9 м', 'participants': [{'role': 'Водитель', 'gender': 'Мужской', 'violations': ['Оставление места ДТП', 'Несоответствие скорости конкретным условиям движения'], 'health_status': 'Не пострадал', 'years_of_driving_experience': 13}]}]",0,"[{'role': 'Пешеход', 'gender': 'Мужской', 'violations': [], 'health_status': 'Раненый, находящийся (находившийся) на стационарном лечении'}]",1,Новгородская область,Сухое,2,"Все участники, Пешеходы","{'type': 'Point', 'coordinates': [30.494614, 58.295429]}"
3,156238,Дорожно-транспортные происшествия,"В темное время суток, освещение включено","{'lat': 58.210136, 'long': 30.721529}","Административные здания, Многоквартирные жилые дома, Нерегулируемый пешеходный переход",Шимский район,740,"рп Шимск, ул Новгородская, 6",Пасмурно,Наезд на пешехода,2022-12-06 17:05:00,Легкий,"[{'year': 2009, 'brand': 'CHEVROLET', 'color': 'Синий', 'model': 'Cruze', 'category': 'D-класс (средний) до 4,6 м', 'participants': [{'role': 'Водитель', 'gender': 'Мужской', 'violations': ['Непредоставление преимущества в движении пешеходу', 'Другие нарушения ПДД водителем'], 'health_status': 'Не пострадал', 'years_of_driving_experience': 25}]}]",0,"[{'role': 'Пешеход', 'gender': 'Женский', 'violations': [], 'health_status': 'Раненый, находящийся (находившийся) на амбулаторном лечении, либо в условиях дневного стационара'}]",1,Новгородская обл

Проверим уникальные значения по полю `tags`:

In [9]:
data['tags'].unique()

array(['Дорожно-транспортные происшествия'], dtype=object)

Данное поле не несет смысловой нагрузки, удалим его:

In [10]:
data = data.drop(columns='tags')

Пропуски по полю `nearby` заполним заглушкой "unknown":

In [11]:
data['nearby'] = data['nearby'].fillna('unknown')

Проверим уникальные значения по полю `weather`:

In [12]:
data['weather'].unique().tolist()

['Ясно',
 'Пасмурно',
 'Снегопад',
 'Метель',
 'Дождь',
 'Туман',
 'Пасмурно, Дождь',
 'Ясно, Температура выше +30С',
 'Пасмурно, Снегопад',
 'Дождь, Снегопад',
 'Ясно, Температура ниже -30С',
 'Пасмурно, Туман',
 'Ясно, Туман',
 'Дождь, Туман',
 'Ясно, Ураганный ветер',
 'Пасмурно, Метель',
 'Снегопад, Метель',
 'Пасмурно, Температура ниже -30С',
 'Метель, Температура ниже -30С',
 'Ясно, Метель',
 'Пасмурно, Ураганный ветер',
 'Ясно, Дождь',
 'Снегопад, Температура ниже -30С',
 'Метель, Ураганный ветер',
 'Дождь, Температура выше +30С',
 'Туман, Снегопад',
 'Дождь, Ураганный ветер',
 'Дождь, Метель',
 'Туман, Температура ниже -30С',
 'Снегопад, Ураганный ветер',
 'Пасмурно, Температура выше +30С',
 'Туман, Метель',
 'Дождь, Температура ниже -30С']

На основе данного поля можно создать несколько категориальных бинарных признаков, выделяющих факты неблагоприятных погодных явлений, например:
* Снегопад
* Метель
* Дождь
* Туман
* Ураганный ветер

Проверим уникальные значения по полю `road_conditions`:

In [13]:
data['road_conditions'].unique().tolist()

['Сухое',
 'Недостатки зимнего содержания, Заснеженное',
 'Мокрое',
 'Заснеженное',
 'Отсутствие, плохая различимость горизонтальной разметки проезжей части, Мокрое',
 'Недостатки зимнего содержания, Гололедица',
 'Обработанное противогололедными материалами',
 'Сухое, Отсутствие освещения',
 'Отсутствие, плохая различимость горизонтальной разметки проезжей части, Сухое, Дефекты покрытия',
 'Сухое, Неудовлетворительное состояние обочин',
 'Отсутствие, плохая различимость горизонтальной разметки проезжей части, Сухое, Отсутствие дорожных знаков в необходимых местах, Неудовлетворительное состояние обочин, Отсутствие тротуаров (пешеходных дорожек)',
 'Отсутствие, плохая различимость горизонтальной разметки проезжей части, Сухое, Неправильное применение, плохая видимость дорожных знаков, Неудовлетворительное состояние обочин, Отсутствие тротуаров (пешеходных дорожек)',
 'Отсутствие, плохая различимость горизонтальной разметки проезжей части, Сухое',
 'Сухое, Неправильное применение, плохая

Поле содержит большое количество комбинаций описания состояния дорожного покрытия. Посчитаем сколько уникальных характеристик содержится - разобъем описание на части между запятыми и посчитаем количество уникальных характеристик:

In [14]:
unique_combinations = data['road_conditions'].str.split(', ').explode().unique()
unique_count = len(unique_combinations)
unique_count

49

In [15]:
print(f'Получилось {unique_count} уникальных характеристик состояния дорожного покрытия')

Получилось 49 уникальных характеристик состояния дорожного покрытия


Посмотрим какие характерисстики получились:

In [16]:
unique_combinations

array(['Сухое', 'Недостатки зимнего содержания', 'Заснеженное', 'Мокрое',
       'Отсутствие',
       'плохая различимость горизонтальной разметки проезжей части',
       'Гололедица', 'Обработанное противогололедными материалами',
       'Отсутствие освещения', 'Дефекты покрытия',
       'Неудовлетворительное состояние обочин',
       'Отсутствие дорожных знаков в необходимых местах',
       'Отсутствие тротуаров (пешеходных дорожек)',
       'Неправильное применение', 'плохая видимость дорожных знаков',
       'Иные недостатки',
       'Отсутствие элементов обустройства остановочного пункта общественного пассажирского транспорта',
       'Со снежным накатом', 'Низкие сцепные качества покрытия',
       'Неисправное освещение', 'Загрязненное', 'Сужение проезжей части',
       'наличие препятствий',
       'затрудняющих движение транспортных средств',
       'плохая различимость вертикальной разметки', 'Неровное покрытие',
       'Отсутствие дорожных ограждений в необходимых местах',
  

Из данных признаков можно выделить следующие ключевые характеристики для создания отдельных категориальных бинарных прзнаков состояния дорожного покрытия:
* Гололедица, заснеженное, снежный накат
* Отсутствие освещения, неисправное освещение, недостаточное освещение,
* Дефекты покрытия, неровное покрытие
* Неисправность светофора, плохая видимость светофора

Проверим уникальные значения по полю `participant_categories`:

In [17]:
data['participant_categories'].unique()

array(['Все участники, Мотоциклисты', 'Все участники',
       'Все участники, Пешеходы', 'Все участники, Велосипедисты',
       'Все участники, Велосипедисты, Мотоциклисты',
       'Все участники, Пешеходы, Мотоциклисты', '',
       'Все участники, Общ. транспорт',
       'Все участники, Пешеходы, Общ. транспорт', 'Мотоциклисты',
       'Все участники, Пешеходы, Велосипедисты',
       'Все участники, Мотоциклисты, Общ. транспорт',
       'Все участники, Велосипедисты, Общ. транспорт',
       'Все участники, Пешеходы, Велосипедисты, Мотоциклисты', 'Пешеходы',
       'Все участники, Пешеходы, Мотоциклисты, Общ. транспорт',
       'Все участники, Пешеходы, Велосипедисты, Общ. транспорт'],
      dtype=object)

Из данного поля можно выделить следующие категориальные признаки:
* Мотоциклисты
* Пешеходы
* Велосипедисты
* Общ. транспорт

#### 2.1.2 Парсинг полей со словарями

В полях `point`, `vehicles`, `participants` содержатся словари, которые необходимо отдельно обработать.

Распарсим поле `point`

In [18]:
# Преобразование словарей в отдельные столбцы
location_df = data['point'].apply(pd.Series)

# Объединяем с оригинальным DataFrame
data = pd.concat([data.drop(columns=['point']), location_df], axis=1)

Удалим дублирующий столбец с координатами:

In [19]:
data = data.drop(columns='geometry')

Распарсим столбец `vehicles`:

In [20]:
vehicle_df = pd.json_normalize(data.explode('vehicles')['vehicles'])

data = data.drop(columns=['vehicles']).join(vehicle_df, lsuffix='_vehicle')

Распарсим столбец `participants`:

In [21]:
# Разворачиваем список словарей в отдельные строки
exploded_df = data.explode('participants')

# Преобразуем словари в отдельные столбцы
participants_df = exploded_df['participants'].apply(pd.Series)

# Объединяем с оригинальным DataFrame, исключая старое поле participants
data = exploded_df.drop(columns=['participants']).join(participants_df)

Удалим столбец `0`:

In [26]:
data = data.drop(columns=0)

Распарсим столбец `violations`:

In [29]:
data['violations'] = data['violations'].fillna('unknown')

data['violations'] = data['violations'].apply(lambda x: x[0] if len(x) == 1 else ', '.join(x) if isinstance(x, list) else None)

Проверим результат:

In [31]:
data.head(5)

,id,light,nearby,region,scheme,address,weather,category_vehicle,datetime,severity,dead_count,participants_vehicle,injured_count,parent_region,road_conditions,participants_count,participant_categories,lat,long,year,brand,color,model,category,role,gender,violations,health_status,years_of_driving_experience
0,1504730,Светлое время суток,,Шимский район,200,"Медведь-Батецкий, 7 км",Ясно,Столкновение,2025-05-01 12:55:00,С погибшими,1,[],0,Новгородская область,Сухое,3,"Все участники, Мотоциклисты",58.332904,30.415138,2012.0,KIA,Серый,Venga,"В-класс (малый) до 3,9 м",Водитель,Женский,Выезд на полосу встречного движения,Не пострадал,4.0
0,1504730,Светлое время суток,,Шимский район,200,"Медведь-Батецкий, 7 км",Ясно,Столкновение,2025-05-01 12:55:00,С погибшими,1,[],0,Новгородская область,Сухое,3,"Все участники, Мотоциклисты",58.332904,30.415138,2012.0,KIA,Серый,Venga,"В-класс (малый) до 3,9 м",Пассажир,Женский,,Не пострадал,NaN
0,1504730,Светлое время суток,,Шимский район,200,"Медведь-Батецкий, 7 км",Ясно,Столкновение,2025-05-01 12:55:00,С погибшими,1,[],0,Новгородская область,Сухое,3,"Все участники, Мотоциклисты",58.332904,30.415138,2012.0,KIA,Серый,Venga,"В-класс (малый) до 3,9 м",Водитель,Женский,Выезд на полосу встречного движения,Не пострадал,4.0
0,1504730,Светлое время суток,,Шимский район,200,"Медведь-Батецкий, 7 км",Ясно,Столкновение,2025-05-01 12:55:00,С погибшими,1,[],0,Новгородская область,Сухое,3,"Все участники, Мотоциклисты",58.332904,30.415138,2012.0,KIA,Серый,Venga,"В-класс (малый) до 3,9 м",Пассажир,Женский,,Не пострадал,NaN
1,156565,Светлое время суток,,Шимский район,600,"Батецкий-Медведь, 10 км",Ясно,Опрокидывание,2015-06-11 16:10:00,Легкий,0,[],1,Новгородская область,Сухое,1,Все участники,NaN,NaN,2013.0,Прочие марки мотоциклов,Красный,Прочие марки мотоциклов,Мопеды с двигателем внутреннего сгорания менее 50 см. куб.,Водитель,Мужской,"Управление ТС лицом, не имеющим права на управление ТС",Скончался на месте ДТП до приезда скорой медицинской помощи,NaN


### 2.2. Обработка пропусков данных

In [ ]:
Проверим наличие NAN и NULL в датасете:

In [ ]:
print(20*'==')
print('NULLS')
print(data.isnull().sum())
print(20*'==')
print('NANs')
print(data.isna().sum())
print(20*'==')

In [ ]:
Обнаружены пропуски в трех полях - `scheme` (данное поле не указано в описании данных), `address`, `geometry`.

Проверим какие данные содержатся в поле `scheme`:

In [ ]:
data['scheme'].unique()

In [ ]:
Здесь в основном представлены трехзначные числа в формате строки. Заполним пропуски заглушкой "unknown"

In [ ]:
data['scheme'] = data['scheme'].fillna('unknown')

In [ ]:
Пропуски по полю address также заполним заглушкой:

In [ ]:
data['address'] = data['address'].fillna('unknown')

### 2.3. Обработка дубликатов

Проверим количество дубликатов в датасете:

In [ ]:
print(data.dtypes)

In [ ]:
data.duplicated().sum()

Явных дубликатов не обнаружено.

Проверим наличие неявных дубликатов по категорийным полям.

Проверим уникальные значения по полю `light`:

In [ ]:
data['light'].unique().tolist()

Неявных дубликатов по полю `light` не обнаружено.

### 2.3. Дообогащение данных

#### 2.3.1. Добавление координат ДТП, соответствующим формату данных DataLens

Для построения геоточек на картах в DataLens необходимы данные в формате '[55.75222,37.61556]', где первое число - широта, вторая долгота.

Получим сначала отдельно координаты широты и долготы:

In [ ]:
data['longitude'] = data['geometry'].geometry.x
data['latitude'] = data['geometry'].geometry.y

In [ ]:
data.to_csv('krsk.csv')